In [1]:
# Import the functions from /tools
import math
import os

# You need to choose one of the api's or hf's "request_llm" function here. 
# from tools.hf_open_source_llm_request import request_llm
# from tools.api_request import request_gpt as request_llm
from tools.api_request import request_gemini as request_llm 

from tools.db_detail import db_getdesc
from tools.db_detail import db_getnames
from tools.sql_execute import sqlite_execute as execute
from tqdm import tqdm
import json
import argparse

def sql_evoke(query,db_name):
    result, execution_time ,executable = execute("datasets/cosql_dataset/database/"+db_name+"/"+db_name+".sqlite",query)
    return result 

def get_example(db_name):
    sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
    result = sql_evoke(sql_query,db_name)
    column_example=""
    for table_name in result:
        column_example = column_example + table_name[0] + ":\n"
        sql_get_eg = "SELECT * FROM "+ table_name[0] +" LIMIT 3;"
        table_eg = sql_evoke(sql_get_eg,db_name)
        for table_data in table_eg:
            column_example = column_example + '('
            for column_data in table_data: 
                column_example = column_example + str(column_data) +','
            column_example = column_example[:-1] + ')\n'
    return column_example

# print(db_getdesc("car_1"),get_example("car_1"))

In [2]:
from agents.rewriter import Rewriter
from agents.selector import Selector
from agents.detector import Detector
from agents.decomposer import Decomposer
from agents.refiner import Refiner

# creat and name the agents
rewriter = Rewriter("Rewriter")
selector = Selector("Selector")
detector = Detector("Detector")
decomposer = Decomposer("Decomposer")
refiner = Refiner("Refiner")

# input_data = "hey!"
# output_rewriter = rewriter.process_input(input_data)
# output_selector = selector.process_input(input_data)
# output_detector = detector.process_input(input_data)
# output_decomposer = decomposer.process_input(input_data)
# output_refiner = refiner.process_input(input_data)

In [ ]:
def process_json_part(data, output_file):
    for index1,item in enumerate(tqdm(data)):
        # Initialnize messages
        print("Turn "+str(index1)+" ==================================================================")
        db_name = item['db_name']
        previous_QA = ""
        
        
        for index, turn in enumerate(item['turns']):
            
            if turn['isuser']:
                final_output = ""
                output_rewriter = ""
                output_selector = ""
                output_detector = ""
                output_decomposer = ""
                output_refiner = ""
                final_output = ""
                sql_output = ""
                user_question = turn['text']
                question_type = turn['type']

                print("previous_QA " + ":" + previous_QA)
                print("question " + str(index//2) + ":" + user_question)
                
                if index+1<len(item['turns']):

                    print("________Question rewriter________")
                    input_data = {
                        "db_desc": db_getnames(db_name),
                        "evidence": "",
                        "question": user_question,
                        "previous_QA": previous_QA
                    }
                    
                    output_rewriter = rewriter.process_input(input_data)
                    
                    if output_rewriter["improper"] == "YES":
                        final_output = output_rewriter["text"]
                    else:
                        rewritten_question = output_rewriter["text"]
                        print("________Select columns and values________")
                        input_data = {
                            "evidence": "",
                            "db_name": db_name,
                            "db_desc": db_getdesc(db_name),
                            "db_exam": get_example(db_name),
                            "question": rewritten_question
                        }
                        
                        output_selector = selector.process_input(input_data)
                        
                        print("________Question type detect________")
                        input_data = {
                            "evidence": "",
                            "db_name": db_name,
                            "db_desc": db_getdesc(db_name),
                            "mini_schema": output_selector,
                            "question": rewritten_question
                        }
                        
                        output_detector = detector.process_input(input_data)
                        # print("output_detectoroutput_detectoroutput_detector" + output_detector)
                        # The question is Answerable
                        print(output_detector)
                        if output_detector == "YES":
                            
                            print("________Decompose question and solve________")
                            input_data = {
                            "evidence": "",
                            "db_name": db_name,
                            "mini_schema": output_selector,
                            "question": rewritten_question
                            }

                            output_decomposer = decomposer.process_input(input_data)
                            print(output_decomposer)
                            
                            if output_decomposer["executable"]:
                                final_output = output_decomposer["sql"]
                                # sql_output = " Result:" + str(output_decomposer["result"])
                            else:
                                
                                print("________Refines erroneous SQL queries________")
                                input_data = {
                                "evidence": "",
                                "db_name": db_name,
                                "mini_schema": output_selector,
                                "question": rewritten_question,
                                "old_sql": output_decomposer.get("sql"),
                                "log": output_decomposer.get("log")
                                }
                                
                                
                                output_refiner = refiner.process_input(input_data)
                                # print(output_refiner)
                                print(output_refiner)
                                final_output = output_refiner["sql"]
                                
                                # if output_decomposer["executable"]:
                                #     sql_output = " Result:" + str(output_refiner["result"])
                            
                        else:
                            final_output = output_detector
                            
                    # llm record
                    print("\nFINAL Response:")
                    print(final_output)
                    print("————————————————————————————————————————————————————————————")
                    item['turns'][index+1]['predict'] = final_output
                    item['turns'][index+1]['Rewriter'] = output_rewriter
                    item['turns'][index+1]['Selector'] = output_selector
                    item['turns'][index+1]['Detector'] = output_detector
                    item['turns'][index+1]['Decomposer'] = output_decomposer
                    item['turns'][index+1]['Refiner'] = output_refiner
                    
                    # update messages
                    g_ans = ""
                    if item['turns'][index+1]['text']:
                        g_ans = item['turns'][index+1]['text']
                    else:
                        g_ans = item['turns'][index+1]['query']
                    if question_type == "answerable":
                        sql_result = sql_evoke(g_ans,db_name)
                        sql_output = " Result:" + str(sql_result)
                    previous_QA += "\nQ:" + user_question + "\nA:" + g_ans + sql_output + '\n'
                    
                    
        if not os.path.exists(output_file):
            with open(output_file, 'w') as f:
                items = [item]
                json.dump(items, f, indent=4)
                f.write('\n')
        else:
            with open(output_file, 'r') as f:
                try:
                    items = json.load(f)
                except json.JSONDecodeError:
                    print("\033[91mError:The file content is not in valid JSON format\033[0m")
                    # items = []

            if not isinstance(items, list):
                print("\033[91mError:The file content is not in valid JSON format\033[0m")
                # items = []

            items.append(item)

            with open(output_file, 'w') as f:
                json.dump(items, f, indent=4)
                f.write('\n') 

with  open('datasets/MMSQL_test.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
process_json_part(data[2:],'agenttest1010.json')

  0%|                                                                                                                                                                                                                                                                                                           | 0/147 [00:00<?, ?it/s]

Turn 0 ==================================================================
previous_QA :
question 0:Find the first name of the students who permanently live in the country Haiti.
________Question rewriter________
request gemini-1.5-flash
{'candidates': [{'content': {'parts': [{'text': '```json\n{"improper": "NO", "rewritten": "What are the first names of the students who permanently live in Haiti?"}\n```'}], 'role': 'model'}, 'finishReason': 'STOP', 'index': 0, 'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE'}]}], 'usageMetadata': {'promptTokenCount': 921, 'candidatesTokenCount': 32, 'totalTokenCount': 953}}
Success: Operation completed after 0 retries
llm_in

        你是一个问题改写器，需要把历史QA和Result整理成一个确定的当前的问题
         [Instructio

  1%|█▉                                                                                                                                                                                                                                                                                               | 1/147 [00:54<2:11:24, 54.00s/it]

{'candidates': [{'content': {'parts': [{'text': 'The question is unanswerable. The database schema does not contain information about student email addresses. You need to provide a different question or update the database schema to include student email addresses. \n'}], 'role': 'model'}, 'finishReason': 'STOP', 'index': 0, 'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE'}]}], 'usageMetadata': {'promptTokenCount': 985, 'candidatesTokenCount': 37, 'totalTokenCount': 1022}}
Success: Operation completed after 0 retries
The question is unanswerable. The database schema does not contain information about student email addresses. You need to provide a different question or update the database schema to include student email addre

  1%|███▉                                                                                                                                                                                                                                                                                             | 2/147 [01:21<1:32:51, 38.42s/it]

...

{'candidates': [{'content': {'parts': [{'text': 'Subquestion 1: Count the number of distinct cylinder values in the CARS_DATA table.\nSQL: ```sql SELECT COUNT(DISTINCT Cylinders) FROM cars_data```\n\nQuestion Solved. \n'}], 'role': 'model'}, 'finishReason': 'STOP', 'index': 0, 'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE'}]}], 'usageMetadata': {'promptTokenCount': 2056, 'candidatesTokenCount': 41, 'totalTokenCount': 2097}}
Success: Operation completed after 0 retries
{'result': [(5,)], 'sql': 'SELECT COUNT(DISTINCT Cylinders) FROM cars_data', 'executable': True, 'log': ''}

FINAL Response:
SELECT COUNT(DISTINCT Cylinders) FROM cars_data
————————————————————————————————————————————————————————————
Turn 5 ==================

  4%|███████████▊                                                                                                                                                                                                                                                                                     | 6/147 [03:27<1:15:33, 32.15s/it]

{'candidates': [{'content': {'parts': [{'text': '```json\n{"improper": "YES", "answer": "I\'m sorry, I can\'t answer that question. I don\'t have access to information about Arabic translations."}\n```'}], 'role': 'model'}, 'finishReason': 'STOP', 'index': 0, 'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE'}]}], 'usageMetadata': {'promptTokenCount': 1157, 'candidatesTokenCount': 42, 'totalTokenCount': 1199}}
Success: Operation completed after 0 retries
llm_in

        你是一个问题改写器，需要把历史QA和Result整理成一个确定的当前的问题
         [Instruction] 你需要根据evidence和previous QA及current question提炼出当前系统需要解答的完整问题作为输入，如果该问题有任何可能可以通过查询数据库来解答则improper为NO如果无关(比如问候或不可能通过数据库推断的问题或日常问题)则YES The output should be in JSON format.
[Requirements] 你需要把整合全部信息整合成一个信息

  5%|█████████████▊                                                                                                                                                                                                                                                                                   | 7/147 [03:53<1:09:48, 29.92s/it]

{'candidates': [{'content': {'parts': [{'text': 'The question is unanswerable. The database schema does not contain information about which countries are US territories. You would need to provide a list of US territories or modify the database schema to include this information. \n'}], 'role': 'model'}, 'finishReason': 'STOP', 'index': 0, 'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE'}]}], 'usageMetadata': {'promptTokenCount': 231, 'candidatesTokenCount': 41, 'totalTokenCount': 272}}
Success: Operation completed after 0 retries
The question is unanswerable. The database schema does not contain information about which countries are US territories. You would need to provide a list of US territories or modify the database sch